## Scikit-learnで未来の株価を予測する

以下の記事を参考に機械学習を用いた課題に取り組んでみる

#### 機械学習で未来を予測する - scikit-learn の決定木で未来の株価を予測
http://qiita.com/ynakayama/items/6a472e5ebbe9365186bd

「決定木」について詳しく知りたい場合は、以下のサイトを参考にする

https://ja.wikipedia.org/wiki/%E6%B1%BA%E5%AE%9A%E6%9C%A8

### 0.環境設定

インストールされていないパッケージがあれば追加する

```
> pip install numpy
> pip install scikit-learn
> pip install pandas
> pip install jupyter
> pip install lxml
> pip install html5lib
> pip install BeautifulSoup4
```

In [1]:
# Jupyterでデータをブラウザ上にプロットする場合は以下を利用する
%matplotlib inline

In [2]:
# 今回の作業で必要になるpythonのライブラリをimportする
import pandas as pd
import numpy as np
from sklearn import tree

In [3]:
# Warningを表示させないオプション設定
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

### 1.データを取得する

以下のサイトを参考に、株価データを用意する

#### 過去のデータからビッグデータ分析で株価を予測する
http://qiita.com/ynakayama/items/420ebe206e34f9941e51

データ取得のPythonスクリプト（ jpstoc.py）は以下のサイトのものを利用する

https://github.com/ynakayama/sandbox/blob/master/python/pandas/jpstock.py

※入手したスクリプトは、このnotebookと同じディレクトリに配置する

In [4]:
from jpstock import JpStock
jpstock = JpStock()

# 取得する銘柄のコード、データの開始年月日を指定する
stock = '####'
start = '2016.1.1'

#  スクリプトを実行してデータを取得し、csvファイルに保存する
stock_tse = jpstock.get(int(stock), start=start)
stock_tse.to_csv("".join(["stock_", stock, ".csv"]))

### 2.不要なデータを掃除する

Pythonのスクリプトでデータをチェックして削除できると良いが、
テキストエディタ等でデータを確認して作業する

In [5]:
# 2016-01-09,610.0,610.0,598.0,600.9,14792000.0,600.9
# 2016-01-12,,,,,,   ←　不要なデータなので削除する
# 2016-01-13,618.0,612.0,612.0,610.9,15292000.0,750.3

### 3.教師データを作る

取得したデータから学習用のデータを作成する

#### 教師データを作るためのメソッドを用意する

In [6]:
# 教師データを作るためのメソッド
def train_data(arr):
    train_X = []
    train_y = []
#    # 90 日間のデータを学習、 1 日ずつ後ろにずらしていく
#    for i in np.arange(-90, -15):
    # 30 日間のデータを学習、 1 日ずつ後ろにずらしていく
    for i in np.arange(-30, -15):
        s = i + 14 # 14 日間の変化を素性にする
        feature = arr.ix[i:s]
        if feature[-1] < arr[s]: # その翌日、株価は上がったか？
            train_y.append(1) # YES なら 1 を
        else:
            train_y.append(0) # NO なら 0 を
        train_X.append(feature.values)
    # 上げ下げの結果と教師データのセットを返す
    return np.array(train_X), np.array(train_y)

#### 取得しておいた株価のデータをプログラム上にロードして教師データを取り出す（作成する）

pandasのDataFrameを用いる

In [7]:
# csv ファイルからの時系列データ読み込み
csvfile = "".join(["stock_", stock, ".csv"])
df = pd.read_csv(csvfile, index_col=0, parse_dates=True)
#df = df[-90:] # 直近の 90 日間
df = df[-30:] # 直近の 30 日間

In [8]:
# 取得したデータからリターンインデックスを算出する
returns = pd.Series(df['Adj Close']).pct_change() # 騰落率を求める
ret_index = (1 + returns).cumprod() # 累積積を求める
ret_index[0] = 1 # 最初の値を 1.0 にする

# リターンインデックスを出力してみる
# print(ret_index)

In [9]:
# リターンインデックスから教師データを取り出す（作成する）
train_X, train_y = train_data(ret_index)

### 3. 決定木アルゴリズムを用いて学習する

In [10]:
# 決定木のインスタンスを生成
clf = tree.DecisionTreeClassifier()

In [11]:
# 学習させる
clf = clf.fit(train_X, train_y)

### 4.うまく学習したかどうか分類器を試す

In [12]:
# 学習後の予測モデルをテストする
test_y = []
## 過去 90 日間のデータでテストをする
#for i in np.arange(-90, -15):             # [-30, -29, -28 .. -16]
# 過去 30 日間のデータでテストをする
for i in np.arange(-30, -15):             # [-30, -29, -28 .. -16]
    s = i + 14
    # リターンインデックスの全く同じ期間をテストとして分類させてみる
    # ループが進むと、s = 16 -> 15 -> 14 .. と変位するので
    # ret_index.ix[-30,-16] -> ret_index.ix[-29, -15]と、
    # 30日前から15日間隔でリターンインデックスの配列を取り出している
    test_X = ret_index.ix[i:s].values

    # 結果を格納して返す
    result = clf.predict(test_X)
    test_y.append(result[0])

print(train_y) # 期待すべき答え
print(np.array(test_y)) # 分類器が出した予測

[0 0 1 0 1 1 1 0 0 1 0 1 1 1 0]
[0 0 1 0 1 1 1 0 0 1 0 1 1 1 0]


### 5. 学習済のモデルを用いて予測する

In [13]:
# 直近14日間のリターンインデックスを抽出
#data_X = ret_index.ix[range(76,90)].values
data_X = ret_index.ix[range(16,30)].values
# print(data_X)

In [14]:
#  抽出したリターンインデックスから株価が上がる（[1]）か下がる（[0]）か予測
result = clf.predict(data_X)
print(result)

[0]
